In [1]:
import pandas as pd
import tqdm

import re
import requests
from bs4 import BeautifulSoup

import hashlib
import datetime

In [2]:
base_url = 'http://www.curitiba.pr.gov.br/dadosabertos/consulta'

In [3]:
params = {"grupo":"3"}

In [4]:
r = requests.get(base_url, params=params)

In [5]:
page = BeautifulSoup(r.text, 'html.parser')

In [6]:
__EVENTVALIDATION = page.find("input", {'name': '__EVENTVALIDATION'}).get('value')

In [7]:
__EVENTVALIDATION

'/wEdAAxVR8e4EOHzCYXnnzUZ7/Id4mKL5o1QRo/FpCKPmYsWJJ6M545mi+eVxYOsufjJuPs5znhzSWYfXU4ugll/AJGuS7DlCF7QoIFOF8dSWE2wXzkiLQS8NWTPNZDV80Z3AzzphndoyDFnIVED7A3KQw9cs48DT8q4XVH83r8Yw95EbJ8/8wAqn890Z+wGGuGKtYqNvjaF5GeBQ6wPsZcNPGG+vCOATgFUUjl7PbiZupqgZarqqZvleoeig3SW/qIZQc3wuPs3mdP64lfOFtVWuTlBMmAEzA=='

In [8]:
__VIEWSTATE = page.find("input", {'name': '__VIEWSTATE'}).get('value')

In [9]:
__VIEWSTATE

'/wEPDwUJMzc0Mzc1MzA2DxYGHgtzdGF0ZV9ncnVwbwUBMx4IaWRmR3J1cG8CAx4IYnVzY2FQb3JkFgJmD2QWAgIDD2QWAgIBDxYCHgZhY3Rpb24FHi9kYWRvc2FiZXJ0b3MvY29uc3VsdGE/Z3J1cG89MxYOAgIPEA8WBh4NRGF0YVRleHRGaWVsZAUKTm9tZVBhZ2luYR4ORGF0YVZhbHVlRmllbGQFCVVybFBhZ2luYR4LXyFEYXRhQm91bmRnFgQeCW9ua2V5ZG93bgVDaWYoSXNFbnRlcktleShldmVudCkpe1JlZGlyZWNpb25hckFjZXNzb1JhcGlkbyh0aGlzKTtyZXR1cm4gZmFsc2U7fR4Hb25jbGljawUfUmVkaXJlY2lvbmFyQWNlc3NvUmFwaWRvKHRoaXMpOxAVBg5BY2Vzc28gUsOhcGlkbxZTZWNyZXRhcmlhcyBlIMOTcmfDo29zEVNlcnZpw6dvcyBPbi1MaW5lGFNlcnZpw6dvcyBwYXJhIGEgRW1wcmVzYRlTZXJ2acOnb3MgcGFyYSBvIENpZGFkw6NvE1NpdGVzIGRhIFByZWZlaXR1cmEVBgANL3NlY3JldGFyaWFzLxkvbGlzdGEtc2Vydmljb3Mtb24tbGluZS8yES9zZXJ2aWNvcy9lbXByZXNhES9zZXJ2aWNvcy9jaWRhZGFvHC9saXN0YS1zaXRlcy1kYS1wcmVmZWl0dXJhLzUUKwMGZ2dnZ2dnZGQCBA8PZBYCHwcFSXJldHVybiBWYWxpZGFyUGVzcXVpc2EoJ3R4dFBlc3F1aXNhVG9wbycsJ2N0bDAwJGxua1Blc3F1aXNhVG9wbycsIGV2ZW50KTtkAgYPDxYCHgRUZXh0BQ1TZXgsIDEyLzEwLzE4ZGQCBw8PFgIeCEltYWdlVXJsBRV+L2ltYWdlbS90ZW1wby9wYy5naWZkZAIIDw8WAh8JBQQyNcK6ZGQCCQ8PFgIfCQUEMTX

In [10]:
pagination = page.find_all(class_='paginacao2')
pagination = list(pagination[0].text.replace('\n', ''))

In [11]:
pages = ""
for page in pagination:
    data = {
        "__EVENTTARGET": f"ctl00$cphMasterPortal$ucPaginador${page}_pg{page}",
        "__EVENTVALIDATION": __EVENTVALIDATION,
        "__VIEWSTATE": __VIEWSTATE
    }
    
    r = requests.request("POST", base_url, params=params, data=data)
    pages = pages + r.text

In [12]:
md5 = lambda s : hashlib.md5(s.encode('utf-8')).hexdigest()

In [13]:
md5('test')

'098f6bcd4621d373cade4e832627b4f6'

In [14]:
a_tags = BeautifulSoup(pages, 'html.parser').find_all(id='linkArquivo')

In [15]:
print(md5(a_tags[0].text))

1f4845ca75ae633e77bf2742390d64a6


In [16]:
date_regex = re.compile('\d{4}-\d{2}-\d{2}')

In [17]:
date_string = date_regex.search('//multimidia.curitiba.pr.gov.br/dadosabertos/SESPAMedicoUnidadesMunicipaisDeSaude/2018-10-10_Sistema_E-Saude_Medicos_-_Base_de_Dados.csv').group()
date = datetime.date(*map(int, date_string.split('-')))
date


datetime.date(2018, 10, 10)

In [18]:
databases = []
for a_tag in tqdm.tqdm(a_tags):
    database_link = a_tag.get('href')[2:]
    
    if database_link[-4:] == '.csv':
        database_id = md5(a_tag.text)
        database_title = a_tag.text

        database_date_string = date_regex.search(database_link).group()
        database_date = datetime.date(*map(int, database_date_string.split('-')))

        database = {
            'database_id': database_id,
            'database_title': database_title,
            'database_link': database_link,
            'database_date': database_date
        }
        
        databases.append(database)

100%|██████████| 4/4 [00:00<00:00, 8661.44it/s]


In [19]:
databases

[{'database_id': '1f4845ca75ae633e77bf2742390d64a6',
  'database_title': 'Servidores Cargos Encargos - Base de Dados',
  'database_link': 'multimidia.curitiba.pr.gov.br/dadosabertos/ServidoresCargosEncargos/2018-10-05_Servidores_Cargos_Encargos_-_Base_de_Dados.csv',
  'database_date': datetime.date(2018, 10, 5)}]

In [20]:
test = pd.read_csv('http://multimidia.curitiba.pr.gov.br/dadosabertos/ServidoresCargosEncargos/2018-10-05_Servidores_Cargos_Encargos_-_Base_de_Dados.csv', encoding="ISO-8859-1", delimiter=';')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,4,6,7,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
test.fillna('').head()

,NOME,CARGO,CARGO_COMISSAO,MES,ANO,LOTACAO,TOTAL_DESCONTOS,TOTAL_LIQUIDO,TOT_VANTAGENS,MOTIVO,DESCRICAO_EVENTO,VALOR_EVENTO,TIPO_EVENTO
0,----,-----,--------------,---,---,-------,---------------,-------------,-------------,------,----------------,------------,-----------
1,ABDO TARBINE,GESTOR PUBLICO MUNICIPAL I,CARGO EM COMISSÃO,9,2018,SECRETARIA DO GOVERNO MUNICIPAL,2378.81,10897.90,13276.71,,PORTAL TRANSPARÊNCIA - PREVIDÊNCIA PÚBLICA,621.03,DESCONTO
2,ABDO TARBINE,GESTOR PUBLICO MUNICIPAL I,CARGO EM COMISSÃO,9,2018,SECRETARIA DO GOVERNO MUNICIPAL,2378.81,10897.90,13276.71,,PORTAL TRANSPARÊNCIA - IRRF,1757.78,DESCONTO
3,ABDO TARBINE,GESTOR PUBLICO MUNICIPAL I,CARGO EM COMISSÃO,9,2018,SECRETARIA DO GOVERNO MUNICIPAL,2378.81,10897.90,13276.71,,"PORTAL TRANSPARÊNCIA - SUBSÍDIO, FG OU CARGO E...",9957.78,VANTAGEM
4,ABDO TARBINE,GESTOR PUBLICO MUNICIPAL I,CARGO EM COMISSÃO,9,2018,SECRETARIA DO GOVERNO MUNICIPAL,2378.81,10897.90,13276.71,,PORTAL TRANSPARÊNCIA - VANTAGENS EVENTUAIS,3318.93,VANTAGEM
